# Chapter 13: Type Guards and Type Predicates

---

## 13.1 Understanding Type Guards

Type guards are runtime checks that help TypeScript narrow down the specific type of a value within a conditional block. They bridge the gap between JavaScript's dynamic nature and TypeScript's static type system, allowing you to write type-safe code when dealing with union types or unknown values.

**The Problem Type Guards Solve:**

```typescript
// Without type guards, we can't safely access type-specific properties
function processValue(value: string | number | boolean) {
  // TypeScript knows value is string | number | boolean
  // But we can't call string-specific methods
  // value.toUpperCase(); // ❌ Error: Property 'toUpperCase' does not exist on type 'number | boolean'
  
  // We need to narrow the type first
  if (typeof value === "string") {
    // TypeScript now knows value is string
    console.log(value.toUpperCase()); // ✅ OK
  }
}

// Type guards enable:
// 1. Safe property access
// 2. Exhaustive checking
// 3. Type narrowing without manual casting
// 4. Better IDE autocomplete
```

**What Type Guards Do:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Type Guard Flow                                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Input: Union Type (string | number | boolean)                     │
│                                                                     │
│        ┌─────────────────┐                                          │
│        │  Type Guard     │                                          │
│        │  (typeof value  │                                          │
│        │   === "string") │                                          │
│        └────────┬────────┘                                          │
│                 │                                                   │
│      ┌──────────┴──────────┐                                        │
│      ▼                     ▼                                        │
│  ┌─────────┐          ┌──────────┐                                   │
│  │  True   │          │  False  │                                   │
│  │ Branch  │          │ Branch  │                                   │
│  │         │          │         │                                   │
│  │ Type:   │          │ Type:   │                                   │
│  │ string  │          │ number  │                                   │
│  │         │          │ boolean │                                   │
│  └─────────┘          └──────────┘                                   │
│                                                                     │
│   Result: TypeScript narrows the type in each branch                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 13.2 Built-in Type Guards

TypeScript provides several built-in mechanisms for type narrowing that leverage JavaScript runtime operators.

### 13.2.1 `typeof` Type Guard

The `typeof` operator checks primitive types at runtime and narrows types accordingly.

**Using `typeof`:**

```typescript
// typeof works with: string, number, bigint, boolean, symbol, undefined, object, function

function processInput(input: string | number | boolean | undefined) {
  // Check for undefined first (common pattern)
  if (input === undefined) {
    console.log("No input provided");
    return;
  }
  
  // Now input is string | number | boolean
  
  if (typeof input === "string") {
    // TypeScript knows: input is string
    console.log(`String length: ${input.length}`);
    console.log(`Uppercase: ${input.toUpperCase()}`);
    return;
  }
  
  if (typeof input === "number") {
    // TypeScript knows: input is number
    console.log(`Formatted: ${input.toFixed(2)}`);
    console.log(`Is integer: ${Number.isInteger(input)}`);
    return;
  }
  
  // TypeScript knows: input is boolean (exhausted other options)
  console.log(`Boolean value: ${input ? "Yes" : "No"}`);
}

// typeof with objects and null (JavaScript quirk)
function checkValue(value: object | null | string) {
  if (typeof value === "object") {
    // TypeScript knows: value is object | null
    // Because typeof null === "object" in JavaScript!
    
    if (value === null) {
      console.log("It's null");
    } else {
      console.log("It's an object");
    }
  } else if (typeof value === "string") {
    console.log("It's a string");
  }
}

// typeof with functions
function execute(value: (() => void) | string) {
  if (typeof value === "function") {
    // TypeScript knows: value is () => void
    value(); // Can safely call
  } else {
    console.log(value); // TypeScript knows: value is string
  }
}
```

**Limitations of `typeof`:**

```typescript
// typeof only works for primitives
class User {
  constructor(public name: string) {}
}

class Product {
  constructor(public title: string) {}
}

function processEntity(entity: User | Product) {
  // typeof entity === "object" for both
  // Can't distinguish between User and Product with typeof
  
  if (typeof entity === "object") {
    // entity is User | Product (both are objects)
    // Need instanceof or custom type guard
  }
}
```

### 13.2.2 `instanceof` Type Guard

The `instanceof` operator checks if an object is an instance of a specific class, narrowing the type to that class.

**Using `instanceof`:**

```typescript
class Cat {
  constructor(public name: string) {}
  
  meow(): void {
    console.log("Meow!");
  }
}

class Dog {
  constructor(public name: string) {}
  
  bark(): void {
    console.log("Woof!");
  }
  
  fetch(item: string): void {
    console.log(`${this.name} fetched ${item}`);
  }
}

function interactWithPet(pet: Cat | Dog): void {
  // Common property accessible on both
  console.log(`Interacting with ${pet.name}`);
  
  // Type narrowing with instanceof
  if (pet instanceof Cat) {
    // TypeScript knows: pet is Cat
    pet.meow(); // OK - Cat specific
    // pet.fetch("ball"); // ❌ Error: Property 'fetch' does not exist on Cat
  } else {
    // TypeScript knows: pet is Dog (only option left)
    pet.bark(); // OK - Dog specific
    pet.fetch("ball"); // OK - Dog specific
  }
}

// instanceof with inheritance
class Animal {
  move(): void {
    console.log("Moving");
  }
}

class Bird extends Animal {
  fly(): void {
    console.log("Flying");
  }
}

class Fish extends Animal {
  swim(): void {
    console.log("Swimming");
  }
}

function moveAnimal(animal: Animal): void {
  animal.move(); // Available on all animals
  
  if (animal instanceof Bird) {
    // TypeScript knows: animal is Bird
    animal.fly(); // OK
  } else if (animal instanceof Fish) {
    // TypeScript knows: animal is Fish
    animal.swim(); // OK
  }
  // Could also be just Animal (base class)
}
```

**Edge Cases with `instanceof`:**

```typescript
// instanceof doesn't work across iframes/windows
// Each frame has its own global scope

// instanceof doesn't work with primitive wrappers
const str = "hello";
const strObj = new String("hello");

console.log(str instanceof String); // false (primitive)
console.log(strObj instanceof String); // true (object)

// instanceof requires a class constructor, not a type
interface Car {
  brand: string;
}

// Can't use instanceof with interfaces (they don't exist at runtime)
function checkCar(vehicle: Car) {
  // if (vehicle instanceof Car) { // ❌ Error: 'Car' only refers to a type
  // }
}
```

### 13.2.3 `in` Operator Type Guard

The `in` operator checks if a property exists on an object, which can be used to distinguish between object types.

**Using `in`:**

```typescript
// Different object shapes
interface Square {
  kind: "square";
  size: number;
}

interface Rectangle {
  kind: "rectangle";
  width: number;
  height: number;
}

interface Circle {
  kind: "circle";
  radius: number;
}

type Shape = Square | Rectangle | Circle;

function getArea(shape: Shape): number {
  // Check for specific properties
  if ("size" in shape) {
    // TypeScript knows: shape is Square (only Square has 'size')
    return shape.size * shape.size;
  }
  
  if ("width" in shape && "height" in shape) {
    // TypeScript knows: shape is Rectangle
    return shape.width * shape.height;
  }
  
  // TypeScript knows: shape is Circle
  return Math.PI * shape.radius * shape.radius;
}

// in with optional properties
interface User {
  name: string;
  email?: string;
}

function getContact(user: User): string {
  if ("email" in user) {
    // TypeScript knows email exists
    return user.email; // string | undefined (still optional)
  }
  return user.name;
}

// Discriminated property check (preferred pattern)
function getAreaBetter(shape: Shape): number {
  // Check the discriminant property
  if (shape.kind === "square") {
    // TypeScript narrows to Square
    return shape.size * shape.size;
  }
  
  // ... etc
}
```

**Cautions with `in`:**

```typescript
// in checks both own and inherited properties
class Base {
  baseProp = "base";
}

class Derived extends Base {
  derivedProp = "derived";
}

const obj = new Derived();

console.log("derivedProp" in obj); // true (own property)
console.log("baseProp" in obj); // true (inherited property)

// in doesn't narrow if property exists on both types with different types
interface A {
  value: string;
}

interface B {
  value: number;
}

function process(item: A | B) {
  if ("value" in item) {
    // Still A | B, just knows 'value' exists
    // item.value is string | number
  }
}
```

---

## 13.3 User-Defined Type Guards

User-defined type guards are functions that return a boolean and use the `is` operator to tell TypeScript about the type of a value.

### 13.3.1 Type Predicates (`is`)

A type predicate is a return type of the form `parameterName is Type`, which tells TypeScript that the parameter is of the specified type if the function returns true.

**Creating Type Guards:**

```typescript
// Define types
interface Cat {
  type: "cat";
  name: string;
  meow(): void;
}

interface Dog {
  type: "dog";
  name: string;
  bark(): void;
}

type Pet = Cat | Dog;

// User-defined type guard using 'is'
function isCat(pet: Pet): pet is Cat {
  return pet.type === "cat";
}

function isDog(pet: Pet): pet is Dog {
  return pet.type === "dog";
}

// Usage
function interact(pet: Pet): void {
  if (isCat(pet)) {
    // TypeScript knows: pet is Cat
    pet.meow();
    console.log(`Cat's name: ${pet.name}`);
  } else {
    // TypeScript knows: pet is Dog
    pet.bark();
    console.log(`Dog's name: ${pet.name}`);
  }
}

// Type guard with more complex logic
interface Admin {
  role: "admin";
  permissions: string[];
}

interface User {
  role: "user";
  email: string;
}

type Person = Admin | User;

function isAdmin(person: Person): person is Admin {
  // Check multiple properties for robustness
  return person.role === "admin" && "permissions" in person;
}

function checkAccess(person: Person): void {
  if (isAdmin(person)) {
    console.log(`Admin with permissions: ${person.permissions.join(", ")}`);
  } else {
    console.log(`Regular user: ${person.email}`);
  }
}
```

**Type Guard with Arrays:**

```typescript
// Type guard for array filtering
interface File {
  type: "file";
  name: string;
  size: number;
}

interface Folder {
  type: "folder";
  name: string;
  children: (File | Folder)[];
}

type FileSystemItem = File | Folder;

function isFile(item: FileSystemItem): item is File {
  return item.type === "file";
}

function isFolder(item: FileSystemItem): item is Folder {
  return item.type === "folder";
}

// Get only files from a mixed array
function getFiles(items: FileSystemItem[]): File[] {
  // Filter with type guard returns correctly typed array
  return items.filter(isFile);
  // Without type guard: return type would be FileSystemItem[]
}

// Get only folders
function getFolders(items: FileSystemItem[]): Folder[] {
  return items.filter(isFolder);
}

// Recursive function with type guards
function calculateSize(item: FileSystemItem): number {
  if (isFile(item)) {
    return item.size;
  }
  
  // TypeScript knows: item is Folder
  return item.children.reduce(
    (total, child) => total + calculateSize(child),
    0
  );
}
```

### 13.3.2 Assertion Functions (`asserts`)

Assertion functions throw an error if a condition is not met, narrowing types after the function call.

**Creating Assertion Functions:**

```typescript
// Assertion function using 'asserts'
function assertIsString(value: unknown): asserts value is string {
  if (typeof value !== "string") {
    throw new TypeError(`Expected string, got ${typeof value}`);
  }
}

function assertIsNumber(value: unknown): asserts value is number {
  if (typeof value !== "number") {
    throw new TypeError(`Expected number, got ${typeof value}`);
  }
  if (isNaN(value)) {
    throw new TypeError("Expected valid number, got NaN");
  }
}

// Usage
function processValue(value: unknown): void {
  // Before assertion: value is unknown
  
  assertIsString(value);
  
  // After assertion: TypeScript knows value is string
  console.log(value.toUpperCase()); // OK
  console.log(value.length); // OK
  
  // const num: number = value; // ❌ Error: string not assignable to number
}

// Assertion with custom error
function assertDefined<T>(value: T | null | undefined): asserts value is T {
  if (value === null || value === undefined) {
    throw new Error("Value must be defined");
  }
}

function getUserName(user: { name?: string } | null): string {
  assertDefined(user);
  
  // TypeScript knows: user is { name?: string }
  
  assertDefined(user.name);
  
  // TypeScript knows: user.name is string
  return user.name; // OK
}

// Asserts with 'this' type guard (class method)
class MyClass {
  private _initialized = false;
  
  initialize(): void {
    this._initialized = true;
  }
  
  assertInitialized(): asserts this is this & { _initialized: true } {
    if (!this._initialized) {
      throw new Error("Must initialize first");
    }
  }
  
  doSomething(): void {
    this.assertInitialized();
    
    // TypeScript knows _initialized is true
    console.log("Doing something");
  }
}
```

---

## 13.4 Creating Custom Type Guards

Building robust custom type guards requires understanding both runtime validation and TypeScript's type system.

### 13.4.1 Validating Object Shapes

**Shape Validation Type Guards:**

```typescript
// Interface definitions
interface User {
  id: number;
  name: string;
  email: string;
}

interface Product {
  id: number;
  title: string;
  price: number;
}

// Type guard for User
function isUser(obj: unknown): obj is User {
  // Runtime validation
  if (typeof obj !== "object" || obj === null) {
    return false;
  }
  
  const user = obj as Record<string, unknown>;
  
  return (
    typeof user.id === "number" &&
    typeof user.name === "string" &&
    typeof user.email === "string" &&
    user.email.includes("@") // Additional validation
  );
}

// Type guard for Product
function isProduct(obj: unknown): obj is Product {
  if (typeof obj !== "object" || obj === null) {
    return false;
  }
  
  const product = obj as Record<string, unknown>;
  
  return (
    typeof product.id === "number" &&
    typeof product.title === "string" &&
    typeof product.price === "number" &&
    product.price >= 0
  );
}

// Safe API response handler
async function fetchUser(url: string): Promise<User | null> {
  try {
    const response = await fetch(url);
    const data: unknown = await response.json();
    
    if (isUser(data)) {
      return data; // TypeScript knows it's User
    }
    
    console.error("Invalid user data received");
    return null;
  } catch (error) {
    console.error("Fetch failed:", error);
    return null;
  }
}

// Usage with type narrowing
function processEntity(entity: unknown): void {
  if (isUser(entity)) {
    console.log(`User: ${entity.name} (${entity.email})`);
  } else if (isProduct(entity)) {
    console.log(`Product: ${entity.title} - $${entity.price}`);
  } else {
    console.log("Unknown entity type");
  }
}
```

### 13.4.2 Generic Type Guards

**Reusable Type Guard Patterns:**

```typescript
// Generic type guard for primitive types
function isOfType<T>(
  value: unknown,
  type: "string" | "number" | "boolean" | "object"
): value is T {
  return typeof value === type;
}

// Usage
const value: unknown = "hello";
if (isOfType<string>(value, "string")) {
  console.log(value.toUpperCase()); // OK
}

// Type guard for array of specific type
function isArrayOf<T>(
  value: unknown,
  itemGuard: (item: unknown) => item is T
): value is T[] {
  return Array.isArray(value) && value.every(itemGuard);
}

// Usage
function isString(item: unknown): item is string {
  return typeof item === "number"; // Oops, bug! But type system trusts us
}

const data: unknown = ["a", "b", "c"];
if (isArrayOf(data, isString)) {
  // data is string[]
  data.forEach(s => console.log(s.toUpperCase()));
}

// Type guard with property existence check
function hasProperty<K extends string>(
  obj: unknown,
  key: K
): obj is Record<K, unknown> {
  return typeof obj === "object" && obj !== null && key in obj;
}

function processWithId(obj: unknown): void {
  if (hasProperty(obj, "id")) {
    // obj is Record<"id", unknown>
    console.log(obj.id); // OK
    
    if (typeof obj.id === "number") {
      console.log(`ID: ${obj.id}`);
    }
  }
}
```

---

## 13.5 Type Guard Best Practices

Following best practices ensures your type guards are reliable and maintainable.

**Best Practice Examples:**

```typescript
// 1. Be exhaustive in type guards
type Status = "loading" | "success" | "error";

function isStatus(value: string): value is Status {
  // Check all possible values
  return ["loading", "success", "error"].includes(value as Status);
}

// 2. Validate thoroughly at runtime
interface ApiResponse {
  data: unknown;
  status: number;
}

function isApiResponse(obj: unknown): obj is ApiResponse {
  if (typeof obj !== "object" || obj === null) return false;
  
  const response = obj as Record<string, unknown>;
  
  // Check all required properties
  return (
    "data" in response &&
    "status" in response &&
    typeof response.status === "number"
  );
}

// 3. Use discriminated unions when possible (more reliable than property checks)
type Result<T> =
  | { kind: "success"; value: T }
  | { kind: "error"; error: string };

function isSuccess<T>(result: Result<T>): result is { kind: "success"; value: T } {
  return result.kind === "success";
}

// 4. Combine type guards for complex validation
interface ValidUser {
  name: string;
  age: number;
  email: string;
}

function isValidUser(obj: unknown): obj is ValidUser {
  return (
    isUser(obj) && // Base check
    typeof (obj as any).age === "number" &&
    (obj as any).age >= 0 &&
    (obj as any).age <= 150
  );
}

// 5. Document assumptions
/**
 * Checks if value is a non-empty array
 * @param value - Value to check
 * @returns true if value is an array with at least one element
 */
function isNonEmptyArray<T>(value: T[] | null | undefined): value is [T, ...T[]] {
  return Array.isArray(value) && value.length > 0;
}

// 6. Avoid type guard functions that always return true (unsafe)
// Bad:
function unsafeIsString(value: unknown): value is string {
  return true; // Always returns true - dangerous!
}

// 7. Use type guards in array methods
interface File {
  name: string;
  content: string;
}

function isFile(item: unknown): item is File {
  // Implementation...
  return true;
}

// Filter with type guard automatically narrows type
const files: unknown[] = fetchFiles();
const validFiles = files.filter(isFile); // Type: File[]

// 8. Consider using branded types for extra safety
type UserId = string & { __brand: "UserId" };
type ProductId = string & { __brand: "ProductId" };

function isUserId(value: string): value is UserId {
  return value.startsWith("user-");
}

function isProductId(value: string): value is ProductId {
  return value.startsWith("prod-");
}
```

---

## 13.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored type guards comprehensively:

**Key Takeaways:**

1. **Built-in Type Guards**:
   - `typeof` for primitives (string, number, boolean, etc.)
   - `instanceof` for class instances
   - `in` for property existence
   - Truthiness/falsiness checks

2. **User-Defined Type Guards**:
   - Use `parameter is Type` return type
   - Enable custom validation logic
   - Essential for runtime type checking

3. **Assertion Functions**:
   - Use `asserts parameter is Type`
   - Throw errors to narrow types
   - Useful for pre-condition checks

4. **Best Practices**:
   - Validate thoroughly at runtime
   - Use discriminated unions when possible
   - Be exhaustive in checks
   - Document assumptions
   - Combine type guards for complex validation

### Practical Exercises

**Exercise 1: Basic Type Guards**

Create type guards for a content management system:

```typescript
// 1. Define types:
//    - Article: { type: "article"; title: string; content: string; author: string }
//    - Video: { type: "video"; title: string; url: string; duration: number }
//    - Image: { type: "image"; title: string; src: string; width: number; height: number }

// 2. Create type guard functions:
//    - isArticle(item): item is Article
//    - isVideo(item): item is Video
//    - isImage(item): item is Image

// 3. Create a function processContent(items: unknown[]) that:
//    - Filters valid content items
//    - Returns separate arrays for each type using type guards
//    - Calculates total duration of all videos
```

**Exercise 2: Assertion Functions**

Build validation utilities:

```typescript
// 1. Create assertion functions:
//    - assertIsString(value): asserts value is string
//    - assertIsNumber(value): asserts value is number
//    - assertIsObject(value): asserts value is Record<string, unknown>

// 2. Create a function parseConfig(json: string): Config that:
//    - Parses JSON
//    - Validates structure using assertion functions
//    - Returns typed Config object

// 3. Create assertNever(value: never): never for exhaustiveness checking
//    - Test with a switch statement on a union type
```

**Exercise 3: Generic Type Guards**

Implement reusable validation:

```typescript
// 1. Create a generic type guard:
//    - isArrayOf<T>(value, guard): value is T[]
//    - Takes a type guard function as parameter

// 2. Create a deep type guard for nested objects:
//    - hasProperty<K extends string>(obj, key): obj is Record<K, unknown>
//    - Use it to safely access nested properties

// 3. Create a type guard for API responses:
//    - isApiResponse<T>(obj, dataGuard): obj is { status: number; data: T }
//    - Validates HTTP response structure
```

**Exercise 4: Real-World Application**

Build a type-safe event system:

```typescript
// 1. Define event types:
//    - ClickEvent: { type: "click"; x: number; y: number; target: string }
//    - InputEvent: { type: "input"; value: string; element: string }
//    - SubmitEvent: { type: "submit"; formData: Record<string, string> }

// 2. Create type guards for each event type

// 3. Create a handleEvent(event: unknown) function that:
//    - Validates event is an object
//    - Checks for type property
//    - Uses appropriate type guard based on type
//    - Calls specific handler for each event type
//    - Includes exhaustiveness check

// 4. Create assertion function assertIsValidEvent
```

**Exercise 5: Advanced Patterns**

Implement discriminated union processing:

```typescript
// 1. Define a Result<T, E> type with success and error variants

// 2. Create utility functions:
//    - isSuccess<T, E>(result): result is Success<T>
//    - isFailure<T, E>(result): result is Failure<E>
//    - unwrapOrDefault<T, E>(result, defaultValue): T

// 3. Create a function processResults<T>(results: Result<T, Error>[]) that:
//    - Separates successes from failures using type guards
//    - Returns { successes: T[], failures: Error[] }
//    - Uses assertion functions to ensure all results are handled

// 4. Implement tryCatch wrapper that returns Result type
```

### Additional Resources

- **TypeScript Handbook - Type Guards**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html
- **User-Defined Type Guards**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html#using-type-predicates
- **Assertion Functions**: https://www.typescriptlang.org/docs/handbook/release-notes/typescript-3-7.html#assertion-functions
- **Discriminated Unions**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html#discriminated-unions

---

## Coming Up Next: Chapter 14 - Nullable Types

In the next chapter, we will explore:

- Understanding `null` and `undefined` in TypeScript
- The `strictNullChecks` compiler option
- Optional properties vs nullable types
- Nullish coalescing operator (`??`)
- Optional chaining (`?.`)
- Non-null assertion operator (`!`)
- Definite assignment assertions
- Best practices for handling null and undefined

Nullable types are essential for writing safe TypeScript code, helping you handle the absence of values explicitly and avoid the billion-dollar mistake of null reference errors.

---